In [111]:
import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import os
import csv
import pandas as pd
import numpy
 
sys.path.append(".")

import config

In [112]:
print(config)

<module 'config' from '/Users/dvergarae/Documents/GitHub/MUITSS-ASPY/config.py'>


In [113]:
PROJECT_PATH = config.data_directory

In [114]:
client_credentials_manager = SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [115]:
def create_file(title):
  file_path = '{}/{}.txt'.format(PROJECT_PATH, title)
  if not os.path.exists(file_path):
      result = requests.get("https://www.azlyrics.com/lyrics/{}.html".format(title))
      c = result.content
      soup = BeautifulSoup(c)
      ringtones = soup.find_all("div", class_="ringtone")
        
      if len(ringtones) > 0:
          ringtone = ringtones[0]
          html_lyrics = ringtone.find_next_sibling('div')
          cleanText = html_lyrics.get_text().replace('\r','').replace('\n',' ').replace("\'",'"')

          file_path = '{}/{}.txt'.format(PROJECT_PATH, title.replace('/','-'))

          fo = open(file_path, "w")
          fo.write(cleanText)
          fo.close()
          return 1
      else:
          return 0
  else:
    print('File Already Exists')

In [11]:
create_file('wizkhalifa/youngwildfree')

In [116]:
def create_songs_file(name):
    csvData = [['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artist', 'artist_id', 'album', 'album_id', 'album_release_date', 'album_release_date_precision', 'album_total_tracks']]

    with open('{}/{}.csv'.format(PROJECT_PATH, name), 'w', newline = '', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(csvData)

    csvFile.close()

In [44]:
create_songs_file('songs')

In [45]:
def fetch_songs(offset, count, name = 'songs', limit = 50):
    tracks = []

    with open('{}/{}.csv'.format(PROJECT_PATH, name), 'a', newline = '', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        for i in range(offset, count * limit, limit):
            track_results = sp.search(q='year:1990', type='track', limit=limit,offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                tracks.append([t['id'], t['name'], t['popularity'], t['duration_ms'], t['explicit'], t['artists'][0]['name'], t['artists'][0]['id'],  t['album']['name'], t['album']['id'], t['album']['release_date'], t['album']['release_date_precision'], t['album']['total_tracks']])
        writer.writerows(tracks)
    csvFile.close()


In [46]:
fetch_songs(0, 3)

In [122]:
df = pd.read_csv('{}/{}.csv'.format(PROJECT_PATH, 'songs'))

In [118]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,82,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,day,12
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,71,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,day,67
2,6m59VvDUi0UQsB2eZ9wVbH,Poison,69,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,day,10
3,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,72,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,year,12
4,63vL5oxWrlvaJ0ayNaQnbX,Istanbul,74,153813,False,They Might Be Giants,6zB02lwP6L6ZH32nggQiJT,Flood,7FwAtuhhWivxvK4aPgyyUD,1990-01-02,day,19


In [123]:
pattern = r'[^a-z0-9A-z]'
df['name_az'] = df['name'].str.replace(pattern, '').str.lower()
df['artist_az'] = df['artist'].str.replace(pattern, '').str.lower()
df['has_lyrics'] = 0

In [ ]:
for index, row in df.head(20).iterrows():
   print(row['name_az'], row['artist_az'])
   df.loc[index, 'has_lyrics'] = create_file('{}/{}'.format(row['artist_az'], row['name_az']))

thunderstruck acdc
dontgobreakingmyheart eltonjohn
poison bellbivdevoe
maninthebox aliceinchains
istanbul theymightbegiants
canikickit atribecalledquest
morethanwords extreme
whatilikeaboutyou theromantics
cowboysfromhell pantera
ucanttouchthis mchammer
stayinalivefromsaturdaynightfeversoundtrack beegees
thehumptydance digitalunderground


In [125]:
df.head()

,id,name,popularity,duration_ms,explicit,artist,artist_id,album,album_id,album_release_date,album_release_date_precision,album_total_tracks,name_az,artist_az,has_lyrics
0,57bgtoPSgt236HzfBOd8kj,Thunderstruck,82,292880,False,AC/DC,711MCceyCBcFnzjGY4Q7Un,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,1990-09-24,day,12,thunderstruck,acdc,1
1,5pKJtX4wBeby9qIfFhyOJj,Don't Go Breaking My Heart,71,271133,False,Elton John,3PhoLpVuITZKcymswpck5b,To Be Continued...,7iKDBfOFUtg8E8RbuDhiux,1990-11-08,day,67,dontgobreakingmyheart,eltonjohn,1
2,6m59VvDUi0UQsB2eZ9wVbH,Poison,69,261853,False,Bell Biv DeVoe,2zFZiWQJFFshzojycnXoTL,Poison,6H5mxGUWguDjtQ4Uzd8veD,1990-01-01,day,10,poison,bellbivdevoe,1
3,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,72,284426,True,Alice In Chains,64tNsm6TnZe2zpcMVMOoHL,Facelift,5LbHbwejgZXRZAgzVAjkhj,1990,year,12,maninthebox,aliceinchains,1
4,63vL5oxWrlvaJ0ayNaQnbX,Istanbul,74,153813,False,They Might Be Giants,6zB02lwP6L6ZH32nggQiJT,Flood,7FwAtuhhWivxvK4aPgyyUD,1990-01-02,day,19,istanbul,theymightbegiants,0
5,3Ti0GdlrotgwsAVBBugv0I,Can I Kick It?,72,251573,False,A Tribe Called Quest,09hVIj6vWgoCDtT03h8ZCa,People's Instinctive Travels and the Paths of ...,3kV0i1qqudjf0PGawJ4jck,1990-04-17,day,17,canikickit,atribecalledquest,0
6,1gVgkQFOKa8Wc1HYsJtPdH,More Than Words,70,334106,False,Extreme,6w7j5wQ5AI5OQYlcM15s2L,Extreme II - Pornograffitti,7DKHQxJTI32UyCdDdGwvRC,1990-01-01,day,13,morethanwords,extreme,0
7,6NdcSEhpGGAYXNnnhGS2s6,What I Like About You,70,175400,False,The Romantics,3daM7asS0gCFvyLemNx2EE,What I Like About You (And Other Romantic Hits),5ZwUOFZdWQ81RYMwXc4j3B,1990,year,10,whatilikeaboutyou,theromantics,0
8,2SgbR6ttzoNlCRGQOKjrop,Cowboys From Hell,72,243533,True,Pantera,14pVkFUHDL207LzLHtSA18,Cowboys From Hell,5szY4sBOSD6IFjFN1RtWTe,1990-07-20,day,12,cowboysfromhell,pantera,0
9,1B75hgRqe7A4fwee3g3Wmu,U Can't Touch This,73,257360,False,MC Hammer,2rblp9fJo16ZPTcKDtlmKW,Please Hammer Don't Hurt 'Em,4r1WecJyt5FOhglysp9zhN,1990-02-20,day,13,ucanttouchthis,mchammer,0
